In [ ]:
import os

import SimulateData

In [ ]:
config = {
    "signatures_file_path":     os.path.join('cosmic_signatures', 'COSMIC_v3.4_SBS_GRCh37.txt'),
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'],
    "n_samples":                30,
    "average_noise":            10,
    "save_dir":                 'simulated_data',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':    True,
        'sign_active_prob':         0.4
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg':                      10,
    },
    'counts_distribution': {
        'distribution':             'logscale',
        'min':                      1000,
        'max':                      50000,
    }
}

simulated_data, data_file, config_file = SimulateData.simulate_data(config)
print('')
print(data_file)
print(config_file)
print('')
print(simulated_data.head())